<a href="https://colab.research.google.com/github/10udCryp7/Speech-Practice/blob/main/notebooks/05_Eval_ASR_Whisper_Small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install datasets

In [ ]:
!conda install evaluate

In [ ]:
!conda install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Load Model Pipeline

In [ ]:
from transformers import pipeline

model_id = 'openai/whisper-small'
task = 'automatic-speech-recognition'

pipe = pipeline(task = task, model = model_id)

# Load Dataset

In [ ]:
from datasets import load_dataset

common_voice_test = load_dataset(
    "mozilla-foundation/common_voice_13_0", "dv", split="test"
)

In [ ]:
sub_test = common_voice_test.shuffle(seed = 42).select(range(100))

# Infer

In [ ]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset

prediction_list = []
for prediction in tqdm(
    pipe(
        KeyDataset(sub_test, 'audio'),
        generate_kwargs={"task": "transcribe", "max_new_tokens": 128},
        batch_size=16,
    ),
    total = len(sub_test)
):
  prediction_list.append(prediction['text'])

In [ ]:
labels = [sample['sentence'] for sample in sub_test]

# Evaluate

In [ ]:
from evaluate import load


compute_metric = load('wer')

In [ ]:
wer_score = compute_metric.compute(predictions = prediction_list, references = labels)

wer_score*100

# Eval Norm

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

labels_norm = [normalizer(label) for label in labels]
prediction_list_norm = [normalizer(prediction) for prediction in prediction_list]

wer_score_norm = compute_metric.compute(predictions = prediction_list_norm, references = labels_norm)

wer_score_norm*100